In [9]:
# === SETUP & IMPORTS ===
import carla
import time
import threading
import pandas as pd
import random
import os
import sys
import numpy as np
import cv2
import select

# Ensure output directories exist
os.makedirs('output/video_Auto_pilot', exist_ok=True)

sys.path.append('D:/carla/PythonAPI')
sys.path.append('D:/carla/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent

# === CONNECTION ===
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
map = world.get_map()
blueprints = world.get_blueprint_library()
spawn_points = map.get_spawn_points()

# === CLEANUP OLD ACTORS ===
for a in world.get_actors().filter('*vehicle*'): a.destroy()
for s in world.get_actors().filter('*sensor*'): s.destroy()

In [10]:
# === LABEL SPAWN POINTS ===
print("\U0001F4CD Drawing enlarged label numbers at each spawn point...")
for idx, sp in enumerate(spawn_points):
    loc = sp.location + carla.Location(z=2.5)
    offsets = [(-0.15, 0), (0.15, 0), (0, -0.15), (0, 0.15), (0, 0)]
    for dx, dy in offsets:
        offset_loc = loc + carla.Location(x=dx, y=dy)
        world.debug.draw_string(offset_loc, f"#{idx}", draw_shadow=True,
                                color=carla.Color(r=255, g=255, b=0), life_time=100.0, persistent_lines=True)
print(f"✅ {len(spawn_points)} bold label numbers displayed in simulator.")

# === USER INPUTS ===
start_index = int(input("\nEnter index for START location: "))
spoof_index = int(input("Enter index for SPOOFED GPS location: "))
start_transform = spawn_points[start_index]
spoof_location = spawn_points[spoof_index].location

print("\n=== Free Driving Mode ===")
print(f"Start Location:      {start_transform.location}")
print(f"Spoofed GPS Location:{spoof_location}")

📍 Drawing enlarged label numbers at each spawn point...
✅ 155 bold label numbers displayed in simulator.

=== Free Driving Mode ===
Start Location:      Location(x=-54.344658, y=137.050995, z=0.600000)
Spoofed GPS Location:Location(x=-44.982998, y=114.955147, z=0.600000)


In [11]:
# === VEHICLE ===
vehicle_bp = blueprints.filter('vehicle.lincoln.mkz_2020')[0]
vehicle = world.try_spawn_actor(vehicle_bp, start_transform)
print("\U0001F697 Vehicle spawned.")

spectator = world.get_spectator()
spectator.set_transform(carla.Transform(
    vehicle.get_transform().transform(carla.Location(x=-5, z=2)),
    vehicle.get_transform().rotation))

# === AUTOPILOT ===
vehicle.set_autopilot(True)

# === SPOOFING SETUP ===
spoofing_active = False
spoofed_latitude = spoof_location.x
spoofed_longitude = spoof_location.y
frame_id = 0
video_fps = 20
video_writers = {}
data_records = []
sensor_data = { 'gnss': None, 'imu': None }
sensor_list = []

# === FOLLOW CAMERA ===
def follow_vehicle():
    while vehicle.is_alive:
        transform = vehicle.get_transform()
        spectator.set_transform(carla.Transform(
            transform.transform(carla.Location(x=-6, z=3)),
            transform.rotation))
        time.sleep(0.05)

threading.Thread(target=follow_vehicle, daemon=True).start()

# === SENSOR CALLBACKS ===
def gps_callback(data):
    sensor_data['gnss'] = {
        'latitude': spoofed_latitude if spoofing_active else data.latitude,
        'longitude': spoofed_longitude if spoofing_active else data.longitude
    }

def imu_callback(data):
    sensor_data['imu'] = {
        'accel_x': data.accelerometer.x,
        'accel_y': data.accelerometer.y,
        'accel_z': data.accelerometer.z,
        'gyro_x': data.gyroscope.x,
        'gyro_y': data.gyroscope.y,
        'gyro_z': data.gyroscope.z
    }

def rgb_callback(image):
    global frame_id
    if not (sensor_data['gnss'] and sensor_data['imu']): return

    array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
    rgb_img = array[:, :, :3][:, :, ::-1]

    if 'rgb' not in video_writers:
        path = 'output/video/rgb_video_Auto_pilot.avi'
        video_writers['rgb'] = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'XVID'), video_fps, (image.width, image.height))
    video_writers['rgb'].write(rgb_img)

    velocity = vehicle.get_velocity()
    speed = 3.6 * (velocity.x**2 + velocity.y**2 + velocity.z**2)**0.5

    record = {
        'frame_id': frame_id,
        'image_num': image.frame,
        'latitude': sensor_data['gnss']['latitude'],
        'longitude': sensor_data['gnss']['longitude'],
        'accel_x': sensor_data['imu']['accel_x'],
        'accel_y': sensor_data['imu']['accel_y'],
        'accel_z': sensor_data['imu']['accel_z'],
        'gyro_x': sensor_data['imu']['gyro_x'],
        'gyro_y': sensor_data['imu']['gyro_y'],
        'gyro_z': sensor_data['imu']['gyro_z'],
        'steering_angle': vehicle.get_control().steer,
        'throttle': vehicle.get_control().throttle,
        'brake': vehicle.get_control().brake,
        'speed': speed,
        'label': 'spoofed' if spoofing_active else 'normal'
    }
    data_records.append(record)
    frame_id += 1

def depth_callback(image):
    array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
    depth_img = cv2.applyColorMap(array[:, :, 0], cv2.COLORMAP_JET)

    if 'depth' not in video_writers:
        path = 'output/video/depth_video_Auto_pilot.avi'
        video_writers['depth'] = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'XVID'), video_fps, (image.width, image.height))
    video_writers['depth'].write(depth_img)

def seg_callback(image):
    array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
    seg_img = array[:, :, :3]

    if 'seg' not in video_writers:
        path = 'output/video/segmentation_video_Auto_pilot.avi'
        video_writers['seg'] = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'XVID'), video_fps, (image.width, image.height))
    video_writers['seg'].write(seg_img)

def lidar_callback(point_cloud):
    points = np.frombuffer(point_cloud.raw_data, dtype=np.float32).reshape(-1, 4)[:, :3]
    lidar_img = np.zeros((600, 800, 3), dtype=np.uint8)
    for point in points:
        x = int(point[0] * 2 + 400)
        y = int(-point[1] * 2 + 300)
        if 0 <= x < 800 and 0 <= y < 600:
            lidar_img[y, x] = (0, 255, 0)

    if 'lidar' not in video_writers:
        path = 'output/video/lidar_video_vehicle.avi'
        video_writers['lidar'] = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'XVID'), video_fps, (800, 600))
    video_writers['lidar'].write(lidar_img)

# === SENSOR ATTACHMENT ===
gnss = world.spawn_actor(blueprints.find('sensor.other.gnss'), carla.Transform(), attach_to=vehicle)
imu = world.spawn_actor(blueprints.find('sensor.other.imu'), carla.Transform(), attach_to=vehicle)
camera_bp = blueprints.find('sensor.camera.rgb')
depth_bp = blueprints.find('sensor.camera.depth')
seg_bp = blueprints.find('sensor.camera.semantic_segmentation')
lidar_bp = blueprints.find('sensor.lidar.ray_cast')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))

camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
depth = world.spawn_actor(depth_bp, camera_transform, attach_to=vehicle)
seg = world.spawn_actor(seg_bp, camera_transform, attach_to=vehicle)
lidar = world.spawn_actor(lidar_bp, carla.Transform(carla.Location(x=0, z=2.5)), attach_to=vehicle)

gnss.listen(gps_callback)
imu.listen(imu_callback)
camera.listen(rgb_callback)
depth.listen(depth_callback)
seg.listen(seg_callback)
lidar.listen(lidar_callback)

sensor_list += [gnss, imu, camera, depth, seg, lidar]

🚗 Vehicle spawned.


In [12]:
# === INTERACTIVE SPOOFING LOOP ===
print("\n=== Spoofing Control ===")
print("Type 's' + ENTER → Activate spoofing")
print("Type 'n' + ENTER → Deactivate spoofing")
print("Type 'q' + ENTER → Quit simulation\n")

def spoofing_input_loop():
    global spoofing_active, running
    while running:
        command = input("Command (s=spoof, n=normal, q=quit): ").strip().lower()
        if command == 's':
            spoofing_active = True
            print("🚨 Spoofing Activated.")
        elif command == 'n':
            spoofing_active = False
            print("✅ Spoofing Deactivated.")
        elif command == 'q':
            print("🛑 Quitting simulation.")
            running = False
            break

# === CONTROL LOOP THREAD START ===
running = True
input_thread = threading.Thread(target=spoofing_input_loop)
input_thread.start()

try:
    while running and vehicle.is_alive:
        world.tick()
        time.sleep(0.05)
except KeyboardInterrupt:
    print("🔁 Simulation interrupted by user.")
    running = False

input_thread.join()




=== Spoofing Control ===
Type 's' + ENTER → Activate spoofing
Type 'n' + ENTER → Deactivate spoofing
Type 'q' + ENTER → Quit simulation

🚨 Spoofing Activated.
✅ Spoofing Deactivated.
🚨 Spoofing Activated.
✅ Spoofing Deactivated.
🚨 Spoofing Activated.
✅ Spoofing Deactivated.
🛑 Quitting simulation.


In [ ]:
# === CLEANUP & SAVE ===
try:
    pd.DataFrame(data_records).to_csv('combined_data_Auto_pilot.csv', index=False)
    print("\U0001F4BE combined_data_Auto_pilot.csv saved.")
except Exception as e:
    print(f"Error saving CSV: {e}")

for s in sensor_list:
    try:
        s.stop()
        s.destroy()
    except:
        pass

try:
    vehicle.destroy()
except:
    pass

for writer in video_writers.values():
    try:
        writer.release()
    except:
        pass

print("✅ Data and videos saved. All actors cleaned up.")

💾 combined_data_Auto_pilot.csv saved.
✅ Data and videos saved. All actors cleaned up.


: 